In [295]:
import pandas as pd
import numpy as np
import re

In [296]:
df = pd.read_excel("job-application-dataset.xlsx")
df.head()

,company_id,position_id,date_applied,location,industry,current_status,Unnamed: 6,platform,Unnamed: 8,Unnamed: 9,furthest_status
0,Anthem Blue Cross Blue Shield Nebraska,Data Intern,2023-09-19,NaN,NaN,Rejected,NaN,NaN,NaN,NaN,Rejected
1,Next Era Energy,Data Science Intern,2023-09-19,NaN,NaN,Rejected,NaN,NaN,NaN,NaN,Rejected
2,Wipfli,Data & Analytics Consulting Intern,2023-09-19,NaN,NaN,Rejected,NaN,NaN,NaN,NaN,Rejected
3,Dish,Data Analyst Intern,2023-09-19,NaN,NaN,Rejected,NaN,NaN,NaN,NaN,Rejected
4,Access Holdings,Summer Data Science Analyst Intern,2023-09-20,NaN,NaN,Waiting,NaN,NaN,NaN,NaN,Waiting


In [297]:
# drop unused columns
df = df.drop(columns = ['Unnamed: 6', 'Unnamed: 8', 'Unnamed: 9'], axis = 1)

Creating Companies Table

In [298]:
companies = pd.Series(df['company_id'].str.lower().unique()).to_frame(name = "company_name").reset_index().copy()
companies = companies.rename(columns = {'index' : 'company_id'})
companies.to_csv("Companies.csv", index = False)

Creating Position Table

In [299]:
# convert all positions to lowercase
df['position_id'] = df['position_id'].str.lower()

# convert to most basic role (e.g. data analytics intern, data science intern, data engineer intern, etc.)
def clean_job_title(title):
    # Define mapping rules
    mapping_rules = {
        'analytics|analyst|analysis': 'data analyst intern',
        'science|sciensist': 'data science intern',
        'visualization': 'data visualization intern',
        'engineer|engineering' : 'data engineering intern',
        'machine learning' : 'machine learning intern',
        'data' : 'data intern',
        'business|intelligence|bi' : 'business analyst',
        'tech|technology' : 'technology intern'
    }

    # Check if the title contains any of the substrings and map to the corresponding clean title
    for substrings, clean_title in mapping_rules.items():
        if any(substring in title.lower() for substring in substrings.split('|')):
            return clean_title

    # If no match is found, return the original title
    return title

df['position_id'] = df['position_id'].apply(clean_job_title)


In [300]:
position = pd.Series(df['position_id'].unique()).to_frame(name = 'position').reset_index()
position = position.rename(columns = {'index' : 'position_id'})
position.to_csv('Position.csv', index = False)

Creating Status Table

In [301]:
# convert all to lowercase
df['current_status'] = df['current_status'].str.lower()

# create table
status = pd.Series(df['current_status'].unique()).to_frame(name = "status").reset_index()
status = status.rename(columns = {'index' : 'status_id'})

# manually adding possible positions
extra_status = pd.DataFrame([[5, 'interview 2'], ['6', 'interview 3']], columns = ['status_id', 'status'])

status = pd.concat([status, extra_status], ignore_index=True)
status.to_csv('Status.csv', index = False)

Creating Jobs Table

In [302]:
# map companies to company_id, positions to position_id
df = df.rename(columns = {'company_id' : 'company_name', 'position_id': 'position'})

# convert company_name, current_status, furthest_status to lowercase
df['company_name'] = df['company_name'].str.lower()
df['current_status'] = df['current_status'].str.lower()
df['furthest_status'] = df['furthest_status'].str.lower()

In [303]:
df.columns

Index(['company_name', 'position', 'date_applied', 'location', 'industry',
       'current_status', 'platform', 'furthest_status'],
      dtype='object')

In [304]:
df = df.merge(companies, how = 'left', on = 'company_name')
df = df.drop('company_name', axis = 1)
df = df.merge(position, how = 'left', on = 'position')
df = df.drop('position', axis = 1)
df = df.merge(status, how = 'left', left_on = 'current_status', right_on = 'status')
df = df.rename(columns = {'status_id' : 'current_status_id'})
df = df.drop(['current_status', 'status'], axis = 1)
df = df.merge(status, how = 'left', left_on = 'furthest_status', right_on = 'status')
df = df.rename(columns = {'status_id' : 'furthest_status_id'})
df = df.drop(['furthest_status', 'status'], axis = 1)

In [305]:
df = df.reset_index(names=['index'])

,index,date_applied,location,industry,platform,company_id,position_id,current_status_id,furthest_status_id
0,0,2023-09-19,NaN,NaN,NaN,0,0,0,0
1,1,2023-09-19,NaN,NaN,NaN,1,1,0,0
2,2,2023-09-19,NaN,NaN,NaN,2,2,0,0
3,3,2023-09-19,NaN,NaN,NaN,3,2,0,0
4,4,2023-09-20,NaN,NaN,NaN,4,2,1,1
...,...,...,...,...,...,...,...,...,...
129,129,2023-11-11,"Ewing, NJ",Insurance,Glassdoor,119,2,1,NaN
130,130,2023-11-11,"Chicago, IL",Software,Glassdoor,120,2,1,NaN
131,131,2023-11-11,"Glendale, AZ",Insurance,workday,121,1,1,NaN
132,132,2023-11-12,TBD,Social Media,Company website,122,1,1,NaN


In [308]:
df.to_csv('Jobs.csv')

TO DO: location